In [46]:
import requests

cookies = {
    'JSESSIONID': '195B8317D3E072305A4DE5FAF6F23A04',
    '_ga_SK0P46Z03T': 'GS1.1.1736071923.1.1.1736071963.0.0.0',
    '_ga': 'GA1.3.1066067018.1731827532',
    '_ga_KFJXWPBN67': 'GS1.1.1731827532.1.1.1731827554.0.0.0',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    # 'Cookie': 'JSESSIONID=195B8317D3E072305A4DE5FAF6F23A04; _ga_SK0P46Z03T=GS1.1.1736071923.1.1.1736071963.0.0.0; _ga=GA1.3.1066067018.1731827532; _ga_KFJXWPBN67=GS1.1.1731827532.1.1.1731827554.0.0.0',
    # 'Accept-Encoding': 'gzip, deflate',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.2 Safari/605.1.15',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Priority': 'u=0, i',
}

response = requests.get('http://tools.nlp.itu.edu.tr/IsTurkish', cookies=cookies, headers=headers)

In [31]:
import re
def is_turkish(text):
    # <textarea name="output" class="text_area" wrap="off">zeynep: false\r\n</textarea>
    match = re.search(r'<textarea name="output" class="text_area" wrap="off">(.+): (.+)\r\n</textarea>', text)
    if match:
        return match.group(2) == 'true'
    return False

In [32]:
def check_is_turkish(word):
    data = {
        'input': word
    }
    response = requests.post('http://tools.nlp.itu.edu.tr/IsTurkish', cookies=cookies, headers=headers, data=data, verify=False)
    return is_turkish(response.text)


In [47]:
check_is_turkish("pencil")

False

In [48]:
check_is_turkish("kalemler")

True

In [6]:
kalın_unluler = ["a","ı","o","u"]   
ince_unluler = ["e","i","ö","ü"]

In [38]:
def unlu_dusmesine_ugrayabilirmi(word,kalın_unluler,ince_unluler):
    count = 0
    for i in range(len(word)):
        if word[i] in "aeıioöuü":
            count += 1
    if count ==1:
        return False
    
    kalın = False
    ince = False
    for i in range(len(word)):
        if word[i] in kalın_unluler:
            kalın = True
        if word[i] in ince_unluler:
            ince = True

    if kalın == True and ince == True:
        return False
    
    for i in range(len(word)-1,0,-1):
        if word[i] in "aeıioöuü":
            break
    
    if word[i] not in "ıiuü":
        return False
    
    return True
    

In [37]:
def unlu_dusmesine___ugrat(word):
    for i in range(len(word)-1,0,-1):
            if word[i] in "aeıioöuü":
                break
    nw = word[:i] + word[i+1:] + word[i]
    if check_is_turkish(nw) == True and word[len(word)-1] not in "aeıioöuü":
         return word[:i] + word[i+1:]

In [35]:
kelime = "şekil"

unlu_dusmesine___ugrat(kelime), kelime

('şekl', 'şekil')

In [36]:
kelime = "kalemler"
unlu_dusmesine___ugrat(kelime), kelime

(None, 'kalemler')

In [24]:
import json

with open('kokler_v01.json') as f:
    data = json.load(f)



In [39]:
dictii = {}
for key in list(data.keys()):
    if dictii.get(key):
        continue
    
    dictii[key] = data[key]
    if unlu_dusmesine_ugrayabilirmi(key,kalın_unluler,ince_unluler):
        if unlu_dusmesine___ugrat(key):
            new_key = unlu_dusmesine___ugrat(key)
            dictii[new_key] = data[key]
    else:
        continue    
        
    


In [40]:
with open ('unlu_dusmeli_v02.json','w') as f:
    json.dump(dictii,f,ensure_ascii=False)


In [12]:
data["şekil"]

44